# Running plantseg
## import

In [ ]:
from plantseg.tasks.io_tasks import import_image_task
from pathlib import Path

'''
The Plantseg documentation can be found at:
https://github.com/kreshuklab/plant-seg/tree/master
https://kreshuklab.github.io/plant-seg/
'''

# Transform image into PlantsegImage
path=Path(r"/mnt/e/PROJECTS-01/Adrian/Bachelor_project/Raw_data/t0001_ch1.tif")
semantic_type="raw" # options are 'raw', 'segmentation' and 'prediction'
stack_layout= "ZYX" # options are 'ZYX', 'YX', '2D_time'
image=import_image_task(input_path=path,semantic_type=semantic_type,stack_layout=stack_layout)
raw_image=image #usefull for later tasks


INFO: P [MainThread] 2025-05-27 13:09:09,419 plantseg - Logger configured at initialisation. PlantSeg logger name: plantseg


## Preprocess

In [ ]:
from plantseg.tasks.dataprocessing_tasks import image_cropping_task,set_voxel_size_task,gaussian_smoothing_task
import numpy as np

# Cropping, change intervals to what you need
#[77:195,437:1446,654:]
rectangle_bi= np.array([
    [77, 437, 654],  # Premier coin (z, y, x)
    [77, 1446, 654],  # Deuxième coin (z, y, x)
    [77,1446 ,2304]   # Troisième coin (z, y, x)
])
z_interval_bi=(77,195)

#[82:121,1171:1448,1263:1545]
rectangle_sm=np.array([
    [83, 1171, 1263],  # Premier coin (z, y, x)
    [83, 1448, 1263],  # Deuxième coin (z, y, x)
    [83,1448 ,1545]   # Troisième coin (z, y, x)
])
z_interval_sm=(82,121)

image=image_cropping_task(image=raw_image,rectangle=rectangle_bi,crop_z=z_interval_bi)
rectangle_image=image

# scale image
'''x,y,z pixel size = 0.347,0.347,0.5 microns'''
voxel_size=(0.5,0.347,0.347)
image=set_voxel_size_task(image=image,voxel_size=voxel_size)


# Gaussian smoothing, change sigma
sigma=2.0
image=gaussian_smoothing_task(image=image,sigma=sigma)



## Segmentation

In [ ]:
from plantseg.tasks.prediction_tasks import unet_prediction_task
from plantseg.tasks.segmentation_tasks import dt_watershed_task, clustering_segmentation_task, lmc_segmentation_task

# Segmentation / Prediction
model='generic_light_sheet_3D_unet' #generic_light_sheet_3D_unet or lightsheet_3D_unet_mouse_embryo_cells
model_id=None
device= 'cpu' # cpu or cuda (using cuda makes the hive crash, so use cpu for now)
image=unet_prediction_task(image=image,model_id=model_id,model_name=model,device=device)[-1] # [-1] because returns a list of images, the last one is the prediction


# Prediction with a trained model
config_path=Path("/mnt/e/PROJECTS-01/Adrian/Bachelor_project/code/Train/config.yml")
model_weights_path=Path("/mnt/e/PROJECTS-01/Adrian/Bachelor_project/Train/model/last_checkpoint03.pytorch")
patch=(32, 128, 128) # patch size for the model (same as used for training)
'''
image=unet_prediction_task(
    image=image,
    model_name=None,
    model_id=None,
    config_path=config_path,
    model_weights_path=model_weights_path,
    patch=patch,
    device="cpu")[-1]
'''

prediction_image=image # usefull for later tasks



INFO: P [MainThread] 2025-05-27 13:10:11,955 plantseg.functionals.prediction.prediction - Safari prediction: Running model from custom config path.
INFO: P [MainThread] 2025-05-27 13:10:13,004 plantseg.functionals.prediction.prediction - Computing theoretical minimum halo from model.
INFO: P [MainThread] 2025-05-27 13:10:13,008 plantseg.functionals.prediction.prediction - For raw in shape (118, 1009, 1650): set patch shape (32, 128, 128), set halo shape (92, 92, 92)
INFO: P [MainThread] 2025-05-27 13:10:13,009 plantseg.functionals.prediction.utils.array_predictor - Using batch size of 1 for prediction


100%|██████████| 935/935 [4:49:54<00:00, 18.60s/it]  


### Visualisation

In [ ]:
import napari

viewer, image_layer = napari.imshow(image._data) #image._data is the numpy array of the image (PlantsegImage)
napari.run()

## Watershed task

In [ ]:
from plantseg.tasks.segmentation_tasks import dt_watershed_task

threshold=0.5 # default 0.5
sigma_seeds=0.2 # default 1.0
stacked= False # default False, If True and the image is 3D, processes the image slice-by-slice (2D)
sigma_weights=0 # default 2.0
min_size=100 # default 100
alpha= 0.8 # default 1.0
pixel_pitch= (1.441,1,1) # default None
apply_nonmax_suppression= True # default False
image=dt_watershed_task(image=prediction_image, 
                        threshold=threshold, 
                        sigma_seeds=sigma_seeds,
                        stacked=stacked, 
                        sigma_weights=sigma_weights, 
                        min_size=min_size, 
                        alpha=alpha, 
                        pixel_pitch=pixel_pitch, 
                        apply_nonmax_suppression=apply_nonmax_suppression)


## Clustering task

In [ ]:
from plantseg.tasks.segmentation_tasks import clustering_segmentation_task, lmc_segmentation_task

over_segmentation = image # over-segementation image
mode = 'gasp' # gasp, multicut or mutex_ws
beta = 1.0 # default 0.5
post_min_size = 100 # default 100
image=clustering_segmentation_task(image=rectangle_image,
                                   over_segmentation=over_segmentation,
                                   mode=mode,
                                   beta=beta,
                                   post_min_size=post_min_size)


# Lifted Multicut task if nuclei segmentation is available

boundary_pmap=None # cell boundart prediction (Z,Y,X) normalised
superpixels=None # superpixels/over-segmentation. Must have the same shape as boundary_pmap.
nuclei=None # a nuclear segmentation or prediction map. Must have the same shape as boundary_pmap.
beta=None # default 0.5
post_min_size=None # default 100
#image=lmc_segmentation_task(boundary_pmap: PlantSegImage, superpixels: PlantSegImage, nuclei: PlantSegImage, beta: float = 0.5, post_min_size: int = 100)

/home/adrifern/miniforge3/envs/plant-seg/lib/python3.12/site-packages/elf/segmentation/multicut.py:68: RuntimeWarning: divide by zero encountered in log
  costs = np.log((1. - costs) / costs) + np.log((1. - beta) / beta)


## Postprcessing

In [ ]:
from plantseg.tasks.dataprocessing_tasks import remove_false_positives_by_foreground_probability_task,fix_over_under_segmentation_from_nuclei_task,set_biggest_instance_to_zero_task

# Remove false positive
segmentation=image # input segmentation
foreground=prediction_image # input foreground probability
threshold=0.1 # threshold value , DEfault 0.5 
image=remove_false_positives_by_foreground_probability_task(segmentation=segmentation, 
                                                            foreground=foreground, 
                                                            threshold=threshold)

# Fix Over/Under segmentation 
cell_seg=None # Input cell segmentation 
nuclei_seg=None # Input nuclear segmentation
threshold_merge=None # Threshold for merging cells, as a fraction (0-1)
threshold_split=None # Threshold for splitting cells, as a fraction (0-1)
quantile_min=None # Minimum quantile for filtering nuclei sizes, as a fraction (0-1)
quantile_max=None # Maximum quantile for filtering nuclei sizes, as a fraction (0-1)
boundary=None # Optional boundary probability map for segmentation refinement, default None
image=fix_over_under_segmentation_from_nuclei_task(cell_seg=cell_seg, 
                                                   nuclei_seg=nuclei_seg, 
                                                   threshold_merge=threshold_merge, 
                                                   threshold_split=threshold_merge, 
                                                   quantile_min=quantile_min, 
                                                   quantile_max=quantile_max, 
                                                   boundary=boundary)

# Set biggest object as background task
instance_could_be_zero=False # default False, If True, 0 might be an instance label, add 1 to all labels before processing
image=set_biggest_instance_to_zero_task(image=image,instance_could_be_zero=instance_could_be_zero)


100%|██████████| 239/239 [00:00<00:00, 4023.43it/s]

INFO: P [MainThread] 2025-02-28 10:22:55,471 plantseg.tasks.dataprocessing_tasks - Processing t0001_ch1_cropped_gasp_fg_filtered with shape (39, 277, 282) and max 233, min 0.


## Export


In [ ]:
from plantseg.tasks.io_tasks import export_image_task 

#Export
export_directory=Path(r"E:\PROJECTS-01\Adrian\Bachelor_project\postprocessed_data\gauss25") #  output directory path where the image will be save
name_pattern="t0001_ch1_1" #  default '{file_name}_export', output file name pattern
key=None # default None, key for the image (used only for h5 and zarr formats)
scale_to_origin=True # default True, scale the voxel size to the original one
export_format='tiff' # 'tiff', 'h5' or 'zarr'
data_type='uint16' #  default 'uint16'
export_image_task(image=image,
                  export_directory=export_directory,
                  name_pattern=name_pattern,
                  key=key,
                  scale_to_origin=scale_to_origin,
                  export_format=export_format,
                  data_type=data_type)